In [1]:
def dilution_eq(V_1, M_1, V_2, M_2):
    if V_1 == None:
        volume = (V_2*M_2) / M_1
        return volume
    elif M_2 == None:
        concentration = (V_1*M_1) / V_2
        return round(concentration, 3)

def adjust_take_vol(stock_conc, reac_vol, desired_conc):
    rounding  = 3
    tolerance = 1e-3
    vol_take  = round(dilution_eq(None, stock_conc, reac_vol, desired_conc), rounding)

    while True:
        total_vol    = vol_take + reac_vol
        act_vol_take = dilution_eq(None, stock_conc, total_vol, desired_conc)
        if ( abs(act_vol_take - vol_take) <= tolerance ):
            break
        vol_take = round(act_vol_take, rounding)
    return vol_take

def enzyme_vol(n_per_tube, amount_each_DNA):
    # 1 uL of enzyme per 100 pmol of DNA <=> x*0.01 uL -> x pmol
    return n_per_tube*0.01*amount_each_DNA

print('Number of DNAs per tube?')
n_per_tube = float(input())
print('Amount of each DNA (in pmol)?')
amount_each_DNA = float(input())
print('Take aliquots? (y/n)')
take_aliquots = str(input())

#############################################################
# STEP 1: PHOSPHORYLATION OF TEMPLATE STRAND

# 10 uL -> 100 pmol <=> x*0.1 uL -> x pmol
vol_each_DNA = amount_each_DNA*0.1
total_DNA_vol = n_per_tube*vol_each_DNA

T4_PNK_vol = enzyme_vol(n_per_tube, amount_each_DNA)
tot_reac_vol_1 = total_DNA_vol + T4_PNK_vol
ligase_buffer = adjust_take_vol(10, tot_reac_vol_1, 1)
tot_reac_vol_1 += ligase_buffer
each_DNA_conc_1 = dilution_eq(vol_each_DNA, 10, tot_reac_vol_1, None) # assume stock concentration is 10 uM

print("\nStep 1: Phosphorylation of template strand")
print("\tVolume of DNA per tube: " + str(total_DNA_vol) + " uL (i.e., " + str(vol_each_DNA) + " uL per DNA)")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tVolume of T4 PNK per tube: " + str(T4_PNK_vol) + " uL")
print("\tConcentration of each DNA: " + str(each_DNA_conc_1) + " uM")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_1) + " uL")

#############################################################
# STEP 2: SPLINT ANNEALING

print("\nStep 2: Splint annealing")


print('\nSplint stock concentration (in uM)?')
splint_stock_conc = float(input())

# want 5 fold excess of splints
amount_of_splint_per_DNA = 5*amount_each_DNA
splint_vol_per_DNA = amount_of_splint_per_DNA / splint_stock_conc
splint_vol = n_per_tube*splint_vol_per_DNA

# make ligase buffer concentration 1X again
ligase_buffer = adjust_take_vol(10, splint_vol, 1)

ddH2O = 0
iteration = 0
while iteration <= 1:
    tot_reac_vol_2 = splint_vol + ligase_buffer + tot_reac_vol_1 + ddH2O
    each_DNA_conc_2 = dilution_eq(tot_reac_vol_1, each_DNA_conc_1, tot_reac_vol_2, None)
    splint_conc = dilution_eq(splint_vol_per_DNA, splint_stock_conc, tot_reac_vol_2, None)

    print("\tVolume of splints per tube: " + str(splint_vol) + " uL (5 fold excess)")
    print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
    print("\tConcentration of each DNA: " + str(each_DNA_conc_2) + " uM")
    print("\tConcentration of splints per " + str(amount_each_DNA) + " pmol of each DNA: " + str(splint_conc) + " uM")

    if take_aliquots == 'y':
        aliquot = round((2.5 / each_DNA_conc_2), 2)
        tot_reac_vol_2 -= aliquot
        print("\tTake " + str(aliquot) + " uL for 2.5 pmol aliquot")

    print("\tTotal reaction volume per tube: " + str(tot_reac_vol_2) + " uL")

    if iteration == 0:
        print("Increase reaction volume with dd H2O? (y/n)")
        again = str(input())
        if again == 'y':
            print('What is the desired volume (in uL) of splints + ligase buffer + ddH2O?')
            desired_vol = float(input())

            ligase_buffer = round(dilution_eq(None, 10, desired_vol, 1), 3)
            ddH2O         = desired_vol - ligase_buffer - splint_vol

            print('\tTake ' + str(ddH2O) + ' uL of dd H2O')
        else:
            iteration += 1

    iteration += 1
    

#############################################################
# STEP 3: LIGATION OF ssMC

T4_DNA_lig_vol = enzyme_vol(n_per_tube, amount_each_DNA)

# make ligase buffer concentration 1X again
ligase_buffer = adjust_take_vol(10, T4_DNA_lig_vol, 1)

tot_reac_vol_3 = T4_DNA_lig_vol + ligase_buffer + tot_reac_vol_2
each_DNA_conc_3 = dilution_eq(tot_reac_vol_2, each_DNA_conc_2, tot_reac_vol_3, None)

print("\nStep 3: Ligation of single stranded minicircles")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tVolume of T4 DNA ligase per tube: " + str(T4_DNA_lig_vol) + " uL")
print("\tConcentration of each DNA: " + str(each_DNA_conc_3) + " uM")

if take_aliquots == 'y':
    aliquot = round((2.5 / each_DNA_conc_3), 2)
    tot_reac_vol_3 -= aliquot
    print("\tTake " + str(aliquot) + " uL for 2.5 pmol aliquot")

print("\tTotal reaction volume per tube: " + str(tot_reac_vol_3) + " uL")

#############################################################
# STEP 4: PHOSPHORYLATION OF COMPLEMENTARY STRANDS

# want 2 fold excess of complementary for every 100 pmol of each DNA
amount_of_comp_per_DNA = 2*amount_each_DNA
comp_vol_per_DNA = amount_of_comp_per_DNA*0.1 # assume 10 uM in 10 uL

comp_vol = n_per_tube*comp_vol_per_DNA

T4_PNK_vol = enzyme_vol(n_per_tube, amount_of_comp_per_DNA)

# make ligase buffer concentration 1X again
ligase_buffer = adjust_take_vol(10, comp_vol+T4_PNK_vol, 1)

tot_reac_vol_4 = ligase_buffer + T4_PNK_vol + comp_vol
comp_conc_per_DNA_4 = dilution_eq(comp_vol_per_DNA, 10, tot_reac_vol_4, None)

print("\nStep 4: Phosphorylation of complementary strand")
print("\tVolume of complementary per tube: " + str(comp_vol) + " uL (i.e., " + str(comp_vol_per_DNA) + " uL per DNA; 2 fold excess)")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tVolume of T4 PNK per tube: " + str(T4_PNK_vol) + " uL")
print("\tConcentration of complementary strands per DNA: " + str(comp_conc_per_DNA_4) + " uM")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_4) + " uL")

#############################################################
# STEP 5: COMPLEMENTARY STRAND ANNEALING

tot_reac_vol_5      = tot_reac_vol_3 + tot_reac_vol_4
each_DNA_conc_5     = dilution_eq(tot_reac_vol_3, each_DNA_conc_3, tot_reac_vol_5, None)
comp_conc_per_DNA_5 = dilution_eq(tot_reac_vol_4, comp_conc_per_DNA_4, tot_reac_vol_5, None)

print("\nStep 5: Complementary strand annealing")
print("\tConcentration of each DNA: " + str(each_DNA_conc_5) + " uM")
print("\tConcentration of complementary strands per DNA: " + str(comp_conc_per_DNA_5) + " uM")

if take_aliquots == 'y':
    aliquot = round((2.5 / each_DNA_conc_3), 2)
    tot_reac_vol_5 -= aliquot
    print("\tTake " + str(aliquot) + " uL for 2.5 pmol aliquot")

print("\tTotal reaction volume per tube: " + str(tot_reac_vol_5) + " uL")


#############################################################
# STEP 6: LIGATION OF DOUBLE-STRANDED CIRCLES AND DEACTIVATION

print("\nStep 6: Ligation of the double-stranded circles and deactivation.")

print('\nLigate entire amount? (y/n)?')
option = str(input())
if option == 'y':
    dsDNA_MC_amount = amount_each_DNA
    tot_reac_vol_6  = tot_reac_vol_5
else:
    print('\nAmount to ligate (in pmol)?')
    dsDNA_MC_amount = float(input())
    tot_reac_vol_6  = round(dsDNA_MC_amount / each_DNA_conc_5, 2)
    print("\tTake " + str(tot_reac_vol_6) + " uL for " + str(dsDNA_MC_amount) + " pmol of each DNA")

T4_DNA_lig_vol = enzyme_vol(n_per_tube, dsDNA_MC_amount)

tot_reac_vol_6 += T4_DNA_lig_vol
EDTA_vol = adjust_take_vol(300, tot_reac_vol_6, 30)
tot_reac_vol_6 += EDTA_vol
each_DNA_conc_6 = round(dsDNA_MC_amount / tot_reac_vol_6, 3)

print("\tVolume of T4 DNA ligase per tube: " + str(T4_DNA_lig_vol) + " uL")
print("\tVolume of 300 mM EDTA per tube: " + str(EDTA_vol) + " uL")
print("\tConcentration of each DNA: " + str(each_DNA_conc_6) + " uM")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_6) + " uL")

#############################################################
# STEP 7: BAL 31 REACTION

print("\nStep 7: BAL 31 reaction.")

desired_vol = 10 # uL
ddH2O       = -1

sample_vol      = round(tot_reac_vol_6 / (dsDNA_MC_amount / 1.25), 2)
BAL_nuclease    = 0.5 # uL
tot_reac_vol_7  = sample_vol + BAL_nuclease
BAL_buffer      = round(adjust_take_vol(2, tot_reac_vol_7, 1), 2)
tot_reac_vol_7 += BAL_buffer
EGTA_vol        = adjust_take_vol(300, tot_reac_vol_7, 30)
tot_reac_vol_7 += EGTA_vol

dye_vol         = 2.5 # uL
ddH2O           = round(desired_vol - (tot_reac_vol_7 + dye_vol), 3)

if ddH2O < 0:
    print('More than 10 uL is required.')
    ddH2O   = 0
    dye_vol = adjust_take_vol(4, tot_reac_vol_7, 1)

tot_reac_vol_7 += dye_vol + ddH2O

print("\tVolume of sample per tube: " + str(sample_vol) + " uL")
print("\tVolume of 2X BAL 31 buffer per tube: " + str(BAL_buffer) + " uL")
print("\tVolume of BAL 31 nuclease per tube: " + str(BAL_nuclease) + " uL")
print("\tVolume of 300 mM EGTA per tube: " + str(EGTA_vol) + " uL")
print("\tVolume of 4X Formamide dye per tube: " + str(dye_vol) + " uL")
print("\tVolume of ddH2O per tube: " + str(ddH2O) + " uL")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_7) + " uL")

Number of DNAs per tube?
5
Amount of each DNA (in pmol)?
20
Take aliquots? (y/n)
n

Step 1: Phosphorylation of template strand
	Volume of DNA per tube: 10.0 uL (i.e., 2.0 uL per DNA)
	Volume of 10X T4 DNA ligase buffer per tube: 1.222 uL
	Volume of T4 PNK per tube: 1.0 uL
	Concentration of each DNA: 1.636 uM
	Total reaction volume per tube: 12.222 uL

Step 2: Splint annealing

Splint stock concentration (in uM)?
1000
	Volume of splints per tube: 0.5 uL (5 fold excess)
	Volume of 10X T4 DNA ligase buffer per tube: 0.055 uL
	Concentration of each DNA: 1.565 uM
	Concentration of splints per 20.0 pmol of each DNA: 7.827 uM
	Total reaction volume per tube: 12.777 uL
Increase reaction volume with dd H2O? (y/n)
y
What is the desired volume (in uL) of splints + ligase buffer + ddH2O?
1
	Take 0.4 uL of dd H2O
	Volume of splints per tube: 0.5 uL (5 fold excess)
	Volume of 10X T4 DNA ligase buffer per tube: 0.1 uL
	Concentration of each DNA: 1.512 uM
	Concentration of splints per 20.0 pmol of eac